<a href="https://colab.research.google.com/github/sushant21csu463/RLLAB/blob/main/monte_carlo_policy_iterration_and_value_iterration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
# Define the grid world environment
grid = np.array([
    [0, 0, 0, 1],
    [0, 1, 0, -1],
    [0, 0, 0, 0],
    [1, 0, 1, 0]
])

In [ ]:
# Define possible actions (up, down, left, right)
actions = [(0, -1), (0, 1), (-1, 0), (1, 0)]
# Define the discount factor
gamma = 0.9

In [ ]:
# Initialize the state-value function
state_values = np.zeros(grid.shape)
# Initialize the policy (random policy)
policy = np.ones((*grid.shape, len(actions))) / len(actions)

In [ ]:
# Helper function to check if a state is valid
def is_valid_state(state):
    i, j = state
    return 0 <= i < grid.shape[0] and 0 <= j < grid.shape[1]

In [ ]:
# Helper function to check if a state is terminal
def is_terminal_state(state):
    return grid[state[0], state[1]] in [-1, 1]

In [ ]:
# Helper function to take an action in the environment
def take_action(state, action):
    i, j = state
    di, dj = action
    new_state = (i + di, j + dj)
    return new_state if is_valid_state(new_state) else state

In [ ]:
# Helper function to calculate the expected value for a state
def calculate_expected_value(state, action, state_values):
    i, j = state
    expected_value = 0
    for a in range(len(actions)):
        new_state = take_action(state, actions[a])
        expected_value += policy[i, j, a] * (grid[new_state[0], new_state[1]] + gamma * state_values[new_state[0], new_state[1]])
    return expected_value

In [ ]:
def policy_iteration(episodes):
    for _ in range(episodes):
        episode = []
        state = (2, 0)

        while not is_terminal_state(state):
            action = np.random.choice(len(actions), p=policy[state[0], state[1]])
            new_state = take_action(state, actions[action])
            episode.append((state, action, grid[new_state[0], new_state[1]]))
            state = new_state

        returns = 0
        for t in reversed(range(len(episode))):
            state, action, reward = episode[t]
            returns = gamma * returns + reward
            state_values[state[0], state[1]] += (1 / len(episode)) * (returns - state_values[state[0], state[1]])

    return state_values

In [ ]:
def value_iteration():
    theta = 1e-6
    while True:
        delta = 0
        for i in range(grid.shape[0]):
            for j in range(grid.shape[1]):
                if is_terminal_state((i, j)):
                    continue
                v = state_values[i, j]
                new_state_values = [calculate_expected_value((i, j), a, state_values) for a in range(len(actions))]
                state_values[i, j] = max(new_state_values)
                delta = max(delta, abs(v - state_values[i, j]))
        if delta < theta:
            break

    return state_values

In [ ]:
# Print the results
if __name__ == "__main__":
    print("Policy Iteration:")
    monte_carlo_result = policy_iteration(1000)
    print(monte_carlo_result)

    print("\nValue Iteration:")
    value_iteration_result = value_iteration()
    print(value_iteration_result)

Policy Iteration:
[[ 0.21360175  0.08052634  0.31182021  0.        ]
 [ 0.5159758   0.         -0.22236022  0.        ]
 [ 1.          0.76247237  0.5488324   0.03204074]
 [ 0.          0.93404084  0.          0.56137298]]

Value Iteration:
[[ 0.54972091  0.6500159   0.5781122   0.        ]
 [ 0.69374896  0.          0.23014854  0.        ]
 [ 0.72874831  0.70527322  0.44477025 -0.0697762 ]
 [ 0.          0.84991803  0.          0.42600064]]
